In [5]:
import pandas as pd
from scipy.stats import ttest_ind

In [2]:
df = pd.read_csv('main-results.csv')
df

,model,cifar10,cifar100,caltech101
0,simclr,52.88,26.76,58.38
1,simclr-affine,55.10,28.60,62.34
2,byol,46.11,21.38,51.21
3,byol-affine,52.29,25.86,58.45
4,barlow_twins,50.31,24.48,55.08
5,barlow_twins-affine,50.53,25.35,55.23
6,barlow_twins-affine-investigate-losses,52.81,26.23,58.68


In [3]:
df = df[df['model'] != 'barlow_twins-affine']
df['model'] = df['model'].map(lambda x: 'barlow_twins-affine' if x == 'barlow_twins-affine-investigate-losses' else x)
df

/tmp/ipykernel_9630/1811361098.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['model'] = df['model'].map(lambda x: 'barlow_twins-affine' if x == 'barlow_twins-affine-investigate-losses' else x)


,model,cifar10,cifar100,caltech101
0,simclr,52.88,26.76,58.38
1,simclr-affine,55.10,28.60,62.34
2,byol,46.11,21.38,51.21
3,byol-affine,52.29,25.86,58.45
4,barlow_twins,50.31,24.48,55.08
6,barlow_twins-affine,52.81,26.23,58.68


In [8]:
models = ['simclr', 'byol', 'barlow_twins']
for m in models:
    baseline = df[df['model'] == m].values.ravel()[1:].astype(float)
    affine = df[df['model'] == m + '-affine'].values.ravel()[1:].astype(float)
    perc_inc = ((affine - baseline) / baseline).mean()
    print(m, perc_inc)

simclr 0.05952421237223577
byol 0.16164919687801835
barlow_twins 0.062179438461928214


In [11]:
raw_df = pd.read_csv('main-results-raw.csv')
raw_df = raw_df[raw_df['model'] != 'barlow_twins-affine']
raw_df['model'] = raw_df['model'].map(lambda x: 'barlow_twins-affine' if x == 'barlow_twins-affine-investigate-losses' else x)
raw_df

,model,acc,dataset
0,simclr,0.528700,cifar10
1,simclr,0.529900,cifar10
2,simclr,0.530000,cifar10
3,simclr,0.526600,cifar10
4,simclr,0.528600,cifar10
...,...,...,...
100,barlow_twins-affine,0.593767,caltech101
101,barlow_twins-affine,0.595407,caltech101
102,barlow_twins-affine,0.571350,caltech101
103,barlow_twins-affine,0.583926,caltech101


In [12]:
for model in ['simclr', 'byol', 'barlow_twins']:
    for dataset in raw_df['dataset'].unique():
        subset = raw_df[(raw_df['dataset'] == dataset) & (raw_df['model'].map(lambda x: model in x))]
        baseline = subset.query(f'`model` == "{model}"')['acc'].values
        variant = subset.query(f'`model` == "{model}-affine"')['acc'].values
        ttest = ttest_ind(variant, baseline)
        if ttest.pvalue < 0.05:
            print(len(baseline), len(variant), model, dataset, baseline.mean(), variant.mean(), ttest)
        else:
            print('NOT SIGNIFICANT', len(baseline), len(variant), model, dataset, baseline.mean(), variant.mean(), ttest)

5 5 simclr cifar10 0.5287599999999999 0.55098 Ttest_indResult(statistic=23.64632764029178, pvalue=1.0886671882003824e-08)
5 5 simclr cifar100 0.26759999999999995 0.286 Ttest_indResult(statistic=29.539419757125355, pvalue=1.869529767370436e-09)
5 5 simclr caltech101 0.5838162930563149 0.6234007654455986 Ttest_indResult(statistic=16.19723279916512, pvalue=2.1218593619686708e-07)
5 5 byol cifar10 0.46107999999999993 0.52288 Ttest_indResult(statistic=55.390933617264984, pvalue=1.2520946077281833e-11)
5 5 byol cifar100 0.21381999999999998 0.25861999999999996 Ttest_indResult(statistic=24.905036276677833, pvalue=7.225737672051126e-09)
5 5 byol caltech101 0.5120831055221433 0.5844723892837618 Ttest_indResult(statistic=19.09836116226933, pvalue=5.8522430534607774e-08)
5 5 barlow_twins cifar10 0.50308 0.52812 Ttest_indResult(statistic=19.44183760433507, pvalue=5.088253093428368e-08)
5 5 barlow_twins cifar100 0.24481999999999998 0.26226000000000005 Ttest_indResult(statistic=29.648889480026764, pv

In [10]:
df['model'].unique()

array(['simclr', 'simclr-affine', 'byol', 'byol-affine', 'barlow_twins',
       'barlow_twins-affine'], dtype=object)